In [2]:
import numpy as np
import pandas as pd
import gensim
import pickle
from gensim.models.ldamulticore import LdaModel
from tqdm.auto import tqdm
import re


from utils import cleantext
from nltk.corpus import stopwords
import spacy

/home/lliang06/.local/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [25]:
tfidf = gensim.models.TfidfModel.load('./data/lda/lda68.tfidf')
id2word = pickle.load(open('./data/lda/id2word_sktoken_lemm_bistop.pickle', 'rb'))
lda = LdaModel.load('./data/lda/d100/lda100_66.model')
# lda = LdaModel.load('./data/lda/d100/lda100.model')

In [23]:
lda = LdaModel.load('./data/lda/d100/lda_100.model')

In [7]:
j = 0
data = pickle.load(open('./data/dimension_energy_pi_abstracts_token/dimension_energy_pi_abstracts_token_part' + str(j) + '.pickle', 'rb'))
corpus = [id2word.doc2bow(text) for text in data]
corpus_tfidf = tfidf[corpus]
vec_iter = lda.get_document_topics(corpus_tfidf, minimum_probability = 0)

for i, v in tqdm(enumerate(vec_iter)):
    if (i == 0):
        lda_vec = np.array([w[1] for w in v])
    else:
        lda_vec = np.vstack([lda_vec, np.array([w[1] for w in v])])

with open('./data/lda/d100/embedding/d100_lda_part' + str(j) + '.npy', 'wb') as f:
    np.save(f, lda_vec)

In [8]:
with open('./data/lda/d100/embedding/d100_66_lda_part' + str(j) + '.npy', 'wb') as f:
    np.save(f, lda_vec)

In [9]:
for j in tqdm(range(1, 62)):
    data = pickle.load(open('./data/dimension_energy_pi_abstracts_token/dimension_energy_pi_abstracts_token_part' + str(j) + '.pickle', 'rb'))
    corpus = [id2word.doc2bow(text) for text in data]
    corpus_tfidf = tfidf[corpus]
    vec_iter = lda.get_document_topics(corpus_tfidf, minimum_probability = 0)
    for i, v in tqdm(enumerate(vec_iter)):
        if (i == 0):
            lda_vec = np.array([w[1] for w in v])
        else:
            lda_vec = np.vstack([lda_vec, np.array([w[1] for w in v])])
    with open('./data/lda/d100/embedding/d100_66_lda_part' + str(j) + '.npy', 'wb') as f:
        np.save(f, lda_vec)

In [27]:
df = pd.read_csv('./data/energy_grants_Dimensions_wAbstract_cleanest.csv')

df_clean = df[(df['researcher_id'] != '-999') | (df['researcher_id'] != '-1')][~df['Abstract'].isna()].reset_index(drop = True)
df_clean['abstract_processed'] = df_clean['Abstract'].map(lambda x: cleantext.clean_abstract(x))

abstracts = df_clean[['abstract_processed', 'Award_number']]

additional_stop_words = ['use', 'study', 'result', 'also', 'may', 'find', 'method', 'system',
                         'however', 'suggest', 'paper', 'include', 'increase', 'solution', 
                         'change', 'process', 'different', 'base', 'effect', 'rate', 'Â°']

stop_words = stopwords.words('english') + additional_stop_words

nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])

def remove_lowinfo_words(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in tqdm(texts):
        doc = nlp(sent) 
        texts_out.append([str(token.lemma_) for token in doc 
                          if token.pos_ in allowed_postags 
                          if str(token) not in stop_words 
                          if str(token.lemma_) not in stop_words
                          if len(str(token)) > 1
                          if len(str(token.lemma_)) > 1
                         ])
    return texts_out

unigram = remove_lowinfo_words(abstracts['abstract_processed'].values.tolist())

sents = list(map(' '.join, tqdm(unigram)))
bigram = list(map(lambda x: cleantext.make_ngrams(x.split(), ngram_range = (2, 2), stop_words = stop_words), tqdm(sents)))

bi_stopwords = ['title_introduction', 'section_title', 'title_introduction', 'chapter_present', 'chapter_contain', 
                'contain_section', 'article_doi', 'Â°_c', 'review_ref', 'review_recent', '%_%']

bi_gram = []
for s in tqdm(bigram):
    bi_gram.append(['_'.join(b.split()) for b in s if '_'.join(b.split()) not in bi_stopwords])    

tokens = [i[0] + i[1] for i in tqdm(zip(unigram, bi_gram))]

In [28]:
corpus = [id2word.doc2bow(token) for token in tokens]
corpus_tfidf = tfidf[corpus]

vec_iter = lda.get_document_topics(corpus_tfidf, minimum_probability = 0)

for i, v in tqdm(enumerate(vec_iter)):
    if (i == 0):
        lda_vec = np.array([w[1] for w in v])
    else:
        lda_vec = np.vstack([lda_vec, np.array([w[1] for w in v])])

with open('./data/lda/d100/FOA_d100_66.npy', 'wb') as f:
    np.save(f, lda_vec)